## 1. Cài đặt & Fix Lỗi

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install ultralytics kaggle

import numpy as np
import os
import shutil
import yaml
import cv2
import random
from glob import glob
from tqdm import tqdm
from ultralytics import YOLO
from google.colab import files

print("Nếu có nút Restart Session, hãy bấm nó")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.6 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Nếu có nút Restart Session, hãy bấm nó


## 2. Tải Dataset (Roboflow + MUID-IITR)

In [ ]:
import os
import shutil # Thêm import shutil nếu chưa có
from google.colab import files # Thêm import files nếu chưa có

!mkdir -p /content/raw_datasets
%cd /content/raw_datasets

# 1. Tải Roboflow Datasets (Cơ bản)
datasets_links = [
    # --- USER PROVIDED DATASETS ---
    ("ds_headphones_3", "https://app.roboflow.com/ds/qqqEeSKAlk?key=GT1Xa65onI"), # Headphones 3
    ("ds_headphones_4", "https://app.roboflow.com/ds/cKHwOqmuda?key=qL10KsWlBt"), # Headphones 4
    ("ds_headphones_5", "https://app.roboflow.com/ds/Ailn10pxiU?key=hBaHIe4EfQ"),


    ("ds_paper1", "https://app.roboflow.com/ds/inuabMtp6t?key=jbu7HTlrBf"),       #Paper1
    ("ds_paper2", "https://app.roboflow.com/ds/b4oxAhlW40?key=4A761Kjm5F"),

    ("ds_phone", "https://app.roboflow.com/ds/5ReObgnLbQ?key=HTPSgVzDLW"),        # Phone
    ("ds_phone2", "https://app.roboflow.com/ds/f9k54F7Azq?key=eYssUekSYc"),       # Phone 2

    ("ds_person", "https://app.roboflow.com/ds/PwRwV0c1jL?key=FgXbXeqlpH"),       # Person
]

for name, url in datasets_links:
    if not os.path.exists(name):
        print(f"Đang tải {name}...")
        !mkdir -p {name}
        !curl -L "{url}" > {name}/dataset.zip
        !unzip -q {name}/dataset.zip -d {name}
        !rm {name}/dataset.zip

# 2. Tải Dataset Nâng Cao từ Kaggle (MUID-IITR)
print("\n--- CẤU HÌNH KAGGLE ĐỂ TẢI DATASET 'BẠCH KIM' (MUID-IITR) ---")

if not os.path.exists("/root/.kaggle"):
    !mkdir -p /root/.kaggle

if not os.path.exists("/content/kaggle.json"):
    print("⚠️ Chưa thấy file kaggle.json. Vui lòng upload file kaggle.json ngay bây giờ:")
    uploaded = files.upload()
    for fn in uploaded.keys():
        print(f'Đã nhận file "{fn}" với kích thước {len(uploaded[fn])} bytes')
        !mv {fn} /content/kaggle.json

if os.path.exists("/content/kaggle.json"):
    !cp /content/kaggle.json /root/.kaggle/
    !chmod 600 /root/.kaggle/kaggle.json
    print("✅ Đã xác thực Kaggle thành công!")

    # Tải MUID-IITR (Platinum - Phone Specialist) với ID mới
    print("Đang tải MUID-IITR Dataset (Platinum) với ID mới...")
    muid_target_dir = "/content/raw_datasets/muid"
    os.makedirs(muid_target_dir, exist_ok=True)
    !kaggle datasets download lakshyataragi/mobilephoneusagedatasetiitr -p {muid_target_dir}

    # Giải nén dataset
    zip_file_path = os.path.join(muid_target_dir, "mobilephoneusagedatasetiitr.zip")
    if os.path.exists(zip_file_path):
        print(f"Đang giải nén {zip_file_path}...")
        !unzip -q {zip_file_path} -d {muid_target_dir}
        # Xóa file zip sau khi giải nén để tiết kiệm dung lượng
        os.remove(zip_file_path)
        print("✅ Đã tải và giải nén MUID-IITR thành công!")
    else:
        print(f"⚠️ Không tìm thấy file {zip_file_path} sau khi tải. Vui lòng kiểm tra lại ID dataset hoặc quyền truy cập.")
else:
    print("⚠️ KHÔNG CÓ THÔNG TIN KAGGLE. Sẽ bỏ qua MUID, chỉ dùng Roboflow.")

/content/raw_datasets
Đang tải ds_headphones_3...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   904  100   904    0     0   2729      0 --:--:-- --:--:-- --:--:--  2731
100  896k  100  896k    0     0   697k      0  0:00:01  0:00:01 --:--:-- 1075k
Đang tải ds_headphones_4...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   904  100   904    0     0   2244      0 --:--:-- --:--:-- --:--:--  2248
100 88.8M  100 88.8M    0     0  19.1M      0  0:00:04  0:00:04 --:--:-- 26.3M
Đang tải ds_headphones_5...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   904  100   904    0     0   3445      0 --:--:-- --:--:-- --:--:--  3450
100 91.4M  100 91.4M    0

Saving kaggle.json to kaggle.json
Đã nhận file "kaggle.json" với kích thước 68 bytes
✅ Đã xác thực Kaggle thành công!
Đang tải MUID-IITR Dataset (Platinum) với ID mới...
Dataset URL: https://www.kaggle.com/datasets/lakshyataragi/mobilephoneusagedatasetiitr
License(s): unknown
 98% 595M/610M [00:03<00:00, 170MB/s]
100% 610M/610M [00:03<00:00, 205MB/s]
Đang giải nén /content/raw_datasets/muid/mobilephoneusagedatasetiitr.zip...
✅ Đã tải và giải nén MUID-IITR thành công!


## 3.  AI Auto-Labeling (Teacher: YOLO11x-seg)
Dùng Teacher Model để gán nhãn lại toàn bộ dữ liệu (bao gồm cả MUID).

In [ ]:
# Load model Segmentation Teacher
auto_label_model = YOLO('yolo11x-seg.pt')

def auto_label_dataset(source_path, target_dir, is_video=False):
    print(f"Đang xử lý {source_path}...")
    images = []

    for root, dirs, files in os.walk(source_path):
        for file in files:
            if file.lower().endswith(('.jpg', '.png', '.jpeg')):
                images.append(os.path.join(root, file))

    print(f"   Tổng cộng {len(images)} ảnh để xử lý.")

    # Tạo cấu trúc thư mục Train/Valid
    for split in ['train', 'valid']:
        os.makedirs(f"{target_dir}/{split}/images", exist_ok=True)
        os.makedirs(f"{target_dir}/{split}/labels", exist_ok=True)

    random.shuffle(images)

    for idx, img_path in enumerate(tqdm(images)):
        split = 'train' if idx < len(images) * 0.8 else 'valid'

        file_name = os.path.basename(img_path)
        # Thêm prefix để tránh trùng tên
        prefix = f"{os.path.basename(source_path)}_"
        unique_name = f"{prefix}{file_name}"

        dest_img_path = f"{target_dir}/{split}/images/{unique_name}"
        shutil.copy(img_path, dest_img_path)

        image = cv2.imread(img_path)
        if image is None: continue

        # Run YOLO Segmentation (Teacher)
        results = auto_label_model(image, verbose=False, conf=0.4)[0]
        label_lines = []

        if results.masks:
            for i, mask in enumerate(results.masks.xyn):
                cls_id = int(results.boxes.cls[i])
                # COCO: 0=person, 67=cell phone, 73=book
                # TARGET: 0=person, 1=phone, 2=book, 3=paper
                my_cls_id = -1

                if cls_id == 0:
                    my_cls_id = 0 # person
                elif cls_id == 67:
                    my_cls_id = 1 # phone
                elif cls_id == 73:
                    my_cls_id = 2 # book

                if my_cls_id != -1:
                    mask_str = " ".join(map(str, mask.flatten()))
                    label_lines.append(f"{my_cls_id} {mask_str}")

        if label_lines:
            label_name = unique_name.replace('.jpg', '.txt').replace('.png', '.txt').replace('.jpeg', '.txt')
            with open(f"{target_dir}/{split}/labels/{label_name}", 'w') as f:
                f.write('\n'.join(label_lines))

OUTPUT_DIR = "/content/merged_dataset"
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)

# 1. Auto-label ds1 (Roboflow)
auto_label_dataset("/content/raw_datasets/ds1", OUTPUT_DIR, is_video=False)

# 2. Auto-label MUID (Platinum) - Nếu có
if os.path.exists("/content/raw_datasets/muid"):
    auto_label_dataset("/content/raw_datasets/muid", OUTPUT_DIR, is_video=False)

Đang xử lý /content/raw_datasets/ds1...
   Tổng cộng 0 ảnh để xử lý.


0it [00:00, ?it/s]


Đang xử lý /content/raw_datasets/muid...
   Tổng cộng 888 ảnh để xử lý.


100%|██████████| 888/888 [01:47<00:00,  8.25it/s]


## 4. Merge các Dataset còn lại (ds2 - ds6)

In [ ]:
TARGET_CLASSES = ['person', 'phone', 'material', 'headphones']

CLASS_MAPPING = {
    'person': 'person', 'student': 'person', 'face': 'person', 'head': 'person',
    'phone': 'phone', 'mobile': 'phone', 'cell phone': 'phone', 'telephone': 'phone', 'smartphone': 'phone', 'ProductRecog - v2 2024-11-05 7:03am': 'phone',
    'paper': 'material', 'PAPER': 'material', 'Paper': 'material', 'document': 'material',
    'headphone': 'headphones', 'headphones': 'headphones', 'earphone': 'headphones', 'headset': 'headphones', 'earbuds': 'headphones', 'Headphone': 'headphones', 'ear device': 'headphones', 'left earbud': 'headphones', 'eardevice': 'headphones'
}

os.makedirs(f"{OUTPUT_DIR}/train/images", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/train/labels", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/valid/images", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/valid/labels", exist_ok=True)

def normalize_class(class_name):
    class_name = class_name.lower().strip()
    for key, target in CLASS_MAPPING.items():
        if class_name == key:
            if target == 'ignore': return -1
            return TARGET_CLASSES.index(target)
    return -1

def find_data_yaml(start_dir):
    for root, dirs, files in os.walk(start_dir):
        if 'data.yaml' in files:
            return os.path.join(root, 'data.yaml')
    return None

# Helper function to convert YOLO bounding box (xc yc w h) to segmentation polygon (x1 y1 x2 y2 x3 y3 x4 y4)
def bbox_to_segment(bbox_coords):
    try:
        # bbox_coords is a list of strings: [x_center, y_center, width, height]
        # Convert to float
        xc, yc, w, h = map(float, bbox_coords)
        # Calculate corner points (normalized)
        x1, y1 = xc - w/2, yc - h/2  # Top-left
        x2, y2 = xc + w/2, yc - h/2  # Top-right
        x3, y3 = xc + w/2, yc + h/2  # Bottom-right
        x4, y4 = xc - w/2, yc + h/2  # Bottom-left
        return f"{x1} {y1} {x2} {y2} {x3} {y3} {x4} {y4}"
    except ValueError:
        return None # Return None if conversion fails

good_datasets = datasets_links[0:]

print("🔄 Đang gộp các dataset chuẩn (ds2-ds6) và chuyển đổi sang định dạng segmentation...")

for ds_name, _ in good_datasets:
    ds_path = f"/content/raw_datasets/{ds_name}"
    yaml_path = find_data_yaml(ds_path)

    if not yaml_path:
        print(f"{ds_name}: Không tìm thấy data.yaml -> Bỏ qua.")
        continue

    ds_root = os.path.dirname(yaml_path)
    with open(yaml_path, 'r') as f:
        data_config = yaml.safe_load(f)
    source_classes = data_config.get('names', [])

    for split in ['train', 'valid', 'test']:
        possible_img_dirs = [
            os.path.join(ds_root, split, 'images'),
            os.path.join(ds_root, split),
            os.path.join(ds_path, split, 'images'),
            os.path.join(ds_path, split)
        ]
        src_img_dir = None
        for d in possible_img_dirs:
            if os.path.exists(d) and (glob(f"{d}/*.jpg") or glob(f"{d}/*.png")):
                src_img_dir = d
                break
        if not src_img_dir: continue

        images = glob(f"{src_img_dir}/*.jpg") + glob(f"{src_img_dir}/*.png")
        target_split = 'valid' if split in ['valid', 'test'] else 'train'

        for img_path in tqdm(images, desc=f"{ds_name} - {split}"):
            file_name = os.path.basename(img_path)
            unique_name = f"{ds_name}_{file_name}"
            shutil.copy(img_path, f"{OUTPUT_DIR}/{target_split}/images/{unique_name}")

            label_name = file_name.replace('.jpg', '.txt').replace('.png', '.txt')
            possible_label_paths = [
                img_path.replace('/images/', '/labels/').replace(file_name, label_name),
                os.path.join(ds_root, split, 'labels', label_name),
                os.path.join(ds_path, split, 'labels', label_name)
            ]
            label_path = None
            for p in possible_label_paths:
                if os.path.exists(p):
                    label_path = p
                    break

            if label_path:
                new_labels = []
                with open(label_path, 'r') as lf:
                    for line in lf:
                        parts = line.strip().split()
                        if not parts: continue
                        try:
                            cls_id = int(parts[0])
                            if cls_id < len(source_classes):
                                cls_name = source_classes[cls_id]
                                new_id = normalize_class(cls_name)
                                if new_id != -1:
                                    # Check if the label is a bounding box (5 parts) or segmentation (more than 5 parts)
                                    if len(parts) == 5: # Assuming it's a bounding box: class x_c y_c w h
                                        segment_coords = bbox_to_segment(parts[1:])
                                        if segment_coords:
                                            new_labels.append(f"{new_id} {segment_coords}")
                                    else: # Already segmentation format (class x1 y1 x2 y2 ...)
                                        new_labels.append(f"{new_id} {' '.join(parts[1:])}")
                        except ValueError: continue

                with open(f"{OUTPUT_DIR}/{target_split}/labels/{unique_name.replace('.jpg','.txt').replace('.png','.txt')}", 'w') as nf:
                    nf.write('\n'.join(new_labels))

final_yaml = {
    'path': OUTPUT_DIR,
    'train': 'train/images',
    'val': 'valid/images',
    'nc': len(TARGET_CLASSES),
    'names': TARGET_CLASSES
}

with open(f"{OUTPUT_DIR}/data.yaml", 'w') as f:
    yaml.dump(final_yaml, f)

print("\n Gộp dataset hoàn tất! Model này sẽ CHUYÊN TRỊ VẬT THỂ (Person, Phone, Book, Paper).")

🔄 Đang gộp các dataset chuẩn (ds2-ds6) và chuyển đổi sang định dạng segmentation...


ds_person - train: 100%|██████████| 100/100 [00:00<00:00, 4606.14it/s]


 Gộp dataset hoàn tất! Model này sẽ CHUYÊN TRỊ VẬT THỂ (Person, Phone, Book, Paper).


## 5. Train Model (Object Expert)

In [ ]:
from ultralytics import YOLO

# Load best.pt from previous run for Transfer Learning
model = YOLO('/content/drive/MyDrive/Intelligence-Test-Models/anticheat_production_yolonano/weights/best.pt')

# Train with new classes (Headphones, Material, etc.)
results = model.train(
    data='/content/merged_dataset/data.yaml',
    epochs=100,
    imgsz=640,
    batch=16,  # Reduce if OOM
    patience=20,
    lr0=0.001,
    lrf=0.01,
    warmup_epochs=5,
    augment=True,
    mosaic=1.0,

    # ===== OPTIMIZER =====
    optimizer='AdamW',

    mixup=0.1,
    copy_paste=0.1,
    degrees=10,
    translate=0.1,
    scale=0.5,
    shear=2.0,
    perspective=0.0001,
    flipud=0.0,
    fliplr=0.5,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    project='/content/drive/MyDrive/Intelligence-Test-Models',
    name='anticheat_production_yolonano',
    exist_ok=True,
    device=0,
    verbose=True,
    save=True,
    plots=True,
    val=True,
    save_conf=True,
)

Ultralytics 8.3.240 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/merged_dataset/data.yaml, degrees=10, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=/content/drive/MyDrive/Intelligence-Test-Models/anticheat_production_yolonano/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=anticheat_production_yolonano, nbs=64, nms=False, opse

In [ ]:

from ultralytics import YOLO

model = YOLO(
    '/content/drive/MyDrive/Intelligence-Test-Models/anticheat_production_yolonano/weights/best.pt'
)

model.val(
    data='/content/merged_dataset/data.yaml',
    plots=True,
    save_conf=True,
    project='/content/drive/MyDrive/Intelligence-Test-Models',
    name='anticheat_production_yolonano_val',
    exist_ok=True
)


Ultralytics 8.3.240 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n-seg summary (fused): 113 layers, 2,835,348 parameters, 0 gradients, 9.6 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1796.7±874.6 MB/s, size: 64.5 KB)
val: Scanning /content/merged_dataset/valid/labels... 2747 images, 145 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 2747/2747 1.8Kit/s 1.5s
val: /content/merged_dataset/valid/images/ds_paper1_F_0000_jpg.rf.7154e2bb81c73cdd2961d2508b3712a7.jpg: 1 duplicate labels removed
val: New cache created: /content/merged_dataset/valid/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 172/172 4.1it/s 42.4s
                   all       2747       3099      0.924      0.858       0.92      0.753      0.917      0.844      0.898      0.716
                person        176        315      0.929      0.895       0.96      0.838      0.933     

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x780a52e2fb00>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0

In [2]:
from ultralytics import YOLO

model = YOLO("/content/drive/MyDrive/Intelligence-Test-Models/best.pt")

model.export(
    format="onnx",
    imgsz=640,
    opset=12,           # ✅ Tốt hơn cho ONNX Runtime Web
    simplify=False,     # ✅ Giữ accuracy (nano đã nhẹ rồi)
    dynamic=False,      # ✅ QUAN TRỌNG - fixed size nhanh hơn nhiều
    half=False          # ⚠️ Thêm dòng này - WebGL không hỗ trợ FP16 tốt
)

Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLO11n-seg summary (fused): 113 layers, 2,835,348 parameters, 0 gradients, 9.6 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/Intelligence-Test-Models/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 40, 8400), (1, 32, 160, 160)) (16.8 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0,<2.0.0'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 5 packages in 201ms
Prepared 1 package in 1.08s
Installed 1 package in 342ms
 + onnx==1.20.0

requirements: AutoUpdate success ✅ 2.3s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.20.0 opset 12...
ONNX: export success ✅ 3.6s, saved as '/content/drive/MyDrive/Intelligence-Tes

'/content/drive/MyDrive/Intelligence-Test-Models/best.onnx'